In [ ]:
# Librerias

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pyproj import Proj, transform
import folium

Primero visualizamos el archivo shapefile (.shp) con los caladeros.

In [ ]:
# Ruta al archivo shapefile.
shp_file = r'ruta a caladeros.shp'

# Cargar el archivo shapefile como GeoDataFrame.
gdf = gpd.read_file(shp_file)

# Definir marco de geo-referenciacion (CRS).
crs = 'EPSG:32632'

# Establecer el CRS del GeoDataFrame.
gdf.crs = crs

# Calcular el centroide de cada polígono y transformarlo a latitud y longitud.
gdf['centro'] = gdf['geometry'].centroid
gdf_latlon = gdf.to_crs(epsg=4326)
gdf_latlon['centro_lat'] = gdf_latlon['centro'].apply(lambda x: x.y)
gdf_latlon['centro_lon'] = gdf_latlon['centro'].apply(lambda x: x.x)

# Visualizar el GeoDataFrame.
fig, ax = plt.subplots(figsize=(10, 10))
gdf_latlon.plot(ax=ax, color='blue', edgecolor='black')
plt.title('Centroides de Polígonos')
plt.xlabel('Longitud')
plt.ylabel('Latitud')
plt.grid(True)
plt.show()


Para trabajar con  los caladeros, transformamos el shapefile a a GeoJSON.

In [ ]:
# Ruta del archivo Shapefile.
shp_file = r'ruta a caladeros.shp'

# Cargar el Shapefile como GeoDataFrame.
gdf = gpd.read_file(shp_file)

# Ruta de salida para el archivo GeoJSON.
geojson_file = 'caladeros.geojson'

# Guardar el GeoDataFrame como GeoJSON.
gdf.to_file(geojson_file, driver='GeoJSON')


Obtener las coordenadas de cada polígono.

In [ ]:
# Cargar el GeoJSON como GeoDataFrame.
gdf = gpd.read_file('ruta a caladeros.geojson')

# Crear una lista para almacenar las coordenadas.
coords_list = []

# Iterar sobre cada polígono y extraer sus coordenadas.
for index, row in gdf.iterrows():
    num_poligono = index + 1
    coords = row['geometry'].exterior.coords.xy
    lats = coords[1]
    lons = coords[0]
    for lat, lon in zip(lats, lons):
        coords_list.append([num_poligono, lat, lon])

# Crear un DataFrame con las coordenadas.
df = pd.DataFrame(coords_list, columns=['numero_poligono', 'Latitud', 'Longitud'])

# Guardar el DataFrame como CSV.
df.to_csv('coordenadas_poligonos.csv', index=False)



Convertir las coordenadas UTM a coordenadas geográficas de tipo cartesiano.

In [ ]:
# Cargar el archivo CSV
df = pd.read_csv(r'ruta a coordenadas_poligonos.csv')

# Definir la función para convertir coordenadas UTM a latitud y longitud.
def convertir_a_latlon(utm_x, utm_y):
    try:
        proj_utm = Proj(proj='utm', zone=32, ellps='WGS84')  # Zona 32 corresponde a España en la proyección de Mercator para Europa (WGS84)
        proj_latlon = Proj(proj='latlong', datum='WGS84')
        lon, lat = transform(proj_utm, proj_latlon, utm_x, utm_y)
        return lat, lon
    except Exception as e:
        print(f"Error al convertir las coordenadas UTM a latitud y longitud: {e}")
        return None, None

# Aplicar la función a cada fila del DataFrame.
df[['Latitud', 'Longitud']] = df.apply(lambda row: pd.Series(convertir_a_latlon(row['Longitud'], row['Latitud'])), axis=1)

# Guardar el DataFrame convertido a un nuevo archivo CSV.
df.to_csv('coordenadas_geograficas.csv', index=False)


Truncar el defecto de posición mediante un truncamiento de las coordenadas.

In [ ]:
# Cargar el archivo CSV con las coordenadas.
df = pd.read_csv('ruta a coordenadas_geograficas.csv')

# Función para truncar las coordenadas a 4 dígitos después del punto decimal.
def truncar_coordenadas(coordenada):
    return round(coordenada, 4)

# Aplicar la función a las columnas Latitud y Longitud.
df['Latitud'] = df['Latitud'].apply(truncar_coordenadas)
df['Longitud'] = df['Longitud'].apply(truncar_coordenadas)

# Guardar el DataFrame con las coordenadas truncadas en un nuevo archivo CSV.
df.to_csv('coordenadas_truncadas.csv', index=False)


Ajustar las coordenadas de longitud a su posición real.

In [ ]:
# Cargar el archivo CSV.
df = pd.read_csv('ruta a coordenadas_truncadas.csv')

# Función para reposicionar la longitud.
def restaurar_longitud(longitud):
    return float(longitud) - 12

# Aplicar la función a la columna Longitud.
df['Longitud'] = df['Longitud'].apply(restaurar_longitud)

# Restringir la longitud y latitud a 4 dígitos máximos.
df['Latitud'] = df['Latitud'].apply(lambda x: "{:.4f}".format(round(x, 4)))
df['Longitud'] = df['Longitud'].apply(lambda x: "{:.4f}".format(round(x, 4)))


# Guardar el DataFrame actualizado en un nuevo archivo CSV.
df.to_csv('coordenadas_restauradas.csv', index=False)


Recortar los polígonos exclusivamente a la zona de estudio (Mar de Alborán).

In [ ]:
# Cargar el archivo CSV con las coordenadas truncadas
df = pd.read_csv('ruta a coordenadas_restauradas.csv')

# Filtrar las coordenadas dentro del rango especificado.
df_recortado = df[(df['Latitud'] >= 35.5) & (df['Latitud'] <= 37.5) & (df['Longitud'] >= -5.8) & (df['Longitud'] <= -1)]

# Función para truncar las coordenadas a 4 dígitos después del punto decimal.
def truncar_coordenadas(coordenada):
    return round(coordenada, 4)

# Aplicar la función a las columnas Latitud y Longitud.
df_recortado['Latitud'] = df_recortado['Latitud'].apply(truncar_coordenadas)
df_recortado['Longitud'] = df_recortado['Longitud'].apply(truncar_coordenadas)

# Guardar el DataFrame filtrado en un nuevo archivo CSV.
df_recortado.to_csv('coordenadas_caladeros.csv', index=False)
